In [1]:
%%capture
!pip install -qU langchain chromadb huggingface_hub sentence-transformers pypdf cohere openai tiktoken

In [32]:
import os
from langchain.document_transformers import(
    EmbeddingsClusteringFilter,
    EmbeddingsRedundantFilter)
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.embeddings import HuggingFaceEmbeddings,CohereEmbeddings,HuggingFaceBgeEmbeddings
#alternatively use openaiembeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_transformers import LongContextReorder
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS,Chroma
from langchain.document_loaders import PyPDFLoader


**Cohere API Keys**

In [12]:
import os
from getpass import getpass
import cohere
#get free cohere api key in cohere or openai website
os.environ["COHERE_API_KEY"] = ""

**Setup Embedding Model**

In [20]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                      model_kwargs={"device": 'cuda'},
                                      encode_kwargs = {'normalize_embeddings':False})

hf_bge_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en",
                                             model_kwargs={'device': 'cuda'},
                                             encode_kwargs={'normalize_embeddings':False})
cohere_embeddings = CohereEmbeddings(model="embed-english-light-v3.0")


**Data Processing**

In [21]:
loader_mh = PyPDFLoader("/content/wcms_108221.pdf")
document_mh = loader_mh.load()
print(len(document_mh))
loader_esops = PyPDFLoader("/content/Employee-Stock-Option-Plans-ESOP-Best-Practices-2.pdf")
document_esops = loader_esops.load()
print(len(document_esops))


37
44


**Split Text into Chunks**

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=200)
text_mh = text_splitter.split_documents(document_mh) #mental health
text_esops = text_splitter.split_documents(document_esops)
print(len(text_mh))
print(len(text_esops))

157
50


**Chromadb Indexes Instantiate with different embedding for these two docs**

In [23]:
import chromadb
ABS_PATH = os.path.dirname(os.path.abspath("."))
DB_DIR = os.path.join(ABS_PATH,"db")
#
client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory=DB_DIR,
    anonymized_telemetry=False #moving from local to hub
)
#
mh_vectorstore = Chroma.from_documents(text_mh,
                                       hf_bge_embeddings,
                                       client_settings=client_settings,
                                       collection_name="mental_health",
                                       collection_metadata={"hnsw":"cosine"},
                                       persist_directory='/store/Database')

esops_vectorstore = Chroma.from_documents(text_esops,
                                          hf_embeddings,
                                          client_settings=client_settings,
                                          collection_name="esops",
                                          collection_metadata = {"hnsw":"cosine"},
                                          persist_directory="/store/Database"
)


**Load VectorStore**

In [24]:
retriever_mh = mh_vectorstore.as_retriever(search_type="mmr",
                                           search_kwargs={'k':5,"include_metadata":True})
retriever_esops = esops_vectorstore.as_retriever(search_type="mmr",
                                                 search_kwargs={'k':5,"include_metadata":True})


**Merge all the retrievers**:
This will hold the outputs from both the retrievers and can be used as any other retriever on different types of chains.
The EnsembleRetriever and the MergerRetriever (LOTR) in Langchain do have similar functionalities in that they both combine the results of multiple retrievers. However, they handle the merging of outputs differently.

The EnsembleRetriever uses a technique called Reciprocal Rank Fusion (RRF) to combine the results. It takes a list of retrievers and a corresponding list of weights as input. If no weights are provided, it defaults to equal weighting for all retrievers. The RRF score for each document is calculated as the weight times the reciprocal of the rank plus a constant ‘c’. The final result is a list of items sorted by their weighted RRF scores in descending order

The MergerRetriever (LOTR) merges the outputs from different retrievers in a round-robin fashion. It first gets the relevant documents from all retrievers and then merges them. The merged results will be a list of documents that are relevant to the query and that have been ranked by the different retrievers. Redundant results from the merged retrievers can be removed using the EmbeddingsRedundantFilter with another embedding. The documents can also be divided into clusters or "centers" of meaning, and the closest document to that center can be picked for the final results. This can be done using the EmbeddingsClusteringFilter

As for the specific use case for using the EnsembleRetriever over the MergerRetriever, it depends on your needs.

The EnsembleRetriever can be used to adjust the influence of different retrievers on the final result by assigning different weights to the retrievers.

The MergerRetriever, on the other hand, can be used to reduce the risk of bias in the results and ensure that the most relevant documents are returned first.

In [26]:
lotr = MergerRetriever(retriever=[retriever_mh,retriever_esops])

for chunks in lotr.get_relevant_documents("What is esops?"):
   print(chunks.page_content)



Non-governmen-
tal organisationsare vital in raisingawareness ofmental healthissues, dissemi-nating informa-tion, providingservices, andreducing stigmasurrounding men-tal health prob-lems.Finland provides an example of an extensive occupational health care serv-ice system that reaches 90 % of the workforce.  Occupational health careservices are also responsible for providing rehabilitation services foremployees.  Mental health issues have not traditionally belonged to thedomain of occupational health care services, but their importance to employ-ees’ work ability has been recognised, and more attention and resourceshave been devoted to them.  In the USA and the UK, where the occupation-al health care services operate on a different basis, Employee AssistanceProgrammes (EAP) are becoming a more common and popular means toprovide counselling and confidential information.  EAP services are inde-pendent of but financed by employers.
THE ROLE OF GOVERNMENT AND THE SOCIAL PARTNERS
ESOPS FOR 

In [27]:
for chunks in lotr.get_relevant_documents("What is consequence of Stigma?"):
  print(chunks.page_content)

feelings of shame, fear, and rejection.
4
Stigma surrounds people with mental health difficulties, and the recovery
process is often misunderstood.  Stigmatisation can negatively affect thesuccess of vocational efforts.  For example, it has been reported that manyprofessional workers who either resign a job or take a medical leave relatedto a mental illness episode, such as depression, experience difficulty main-taining a stigma-free relationship with their employers.  Those returning tothe same work environment find that performance and behavioural difficul-ties, which initially interrupted their work, have altered their employers’
and co-workers’ perception of their professional abilities.
5
It is clear that mental health problems can impose a heavy burden in terms
Social Impact Considerations  
•Companies focused on social impact goals and/or in developing 
markets may have unique ESOP considerations  
•Consider two possible scenarios:  
–Financial Inclusion Goals : Employee -friend

**Remove redundant results from merged_retrievers**

In [33]:
filter = EmbeddingsRedundantFilter(embeddings=cohere_embeddings)
pipeline = DocumentCompressorPipeline(transformers=[filter])
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline,
                                                        base_retriever=lotr)


**Retrieve final doc ordered by original retriever scores**

In [34]:
filter_ordered_by_retriever = EmbeddingsClusteringFilter(
    embeddings=cohere_embeddings,
    num_clusters=10,
    num_closest=1,
    sorted=True,
)

pipeline = DocumentCompressorPipeline(transformers=[filter_ordered_by_retriever])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline,base_retriever=lotr
)

**Visualising Results before Reordering results to avoid performance degradation**

In [35]:
lotr = MergerRetriever(retrievers=[retriever_mh,retriever_esops])
query="What is ESOPS?"
docs = lotr.get_relevant_documents(query)
docs

[Document(page_content='Non-governmen-\ntal organisationsare vital in raisingawareness ofmental healthissues, dissemi-nating informa-tion, providingservices, andreducing stigmasurrounding men-tal health prob-lems.Finland provides an example of an extensive occupational health care serv-ice system that reaches 90 % of the workforce.  Occupational health careservices are also responsible for providing rehabilitation services foremployees.  Mental health issues have not traditionally belonged to thedomain of occupational health care services, but their importance to employ-ees’ work ability has been recognised, and more attention and resourceshave been devoted to them.  In the USA and the UK, where the occupation-al health care services operate on a different basis, Employee AssistanceProgrammes (EAP) are becoming a more common and popular means toprovide counselling and confidential information.  EAP services are inde-pendent of but financed by employers.\nTHE ROLE OF GOVERNMENT AND THE 

**Re-ordered Docs(Tackling Lost in the Middle)**

In [36]:
reordering = LongContextReorder()
reordered_docs=reordering.transform_documents(docs)
reordered_docs

[Document(page_content='ESOPS FOR THE LONG TERM  Part IV', metadata={'page': 34, 'source': '/content/Employee-Stock-Option-Plans-ESOP-Best-Practices-2.pdf'}),
 Document(page_content='What is an ESOP?  \n•An Employee Stock Options Plan (ESOP)  \n \n•An allocation  of shares that will be granted to employees in the future \nin the form of stock options  \n–How much equity should we set aside for employees?  \n \n•A plan for how these options will be distributed:  \n–How many shares will individual employees receive?  \n–What terms will govern these grants?  \n \n•The plan is as important as the allocation!', metadata={'page': 3, 'source': '/content/Employee-Stock-Option-Plans-ESOP-Best-Practices-2.pdf'}),
 Document(page_content='Late-VC Flush with capital,  startups at this stage \nbegin to steadily ramp -up hiring, yet \nemployees still want equity  Important to have standardized the  \nESOP and the amount of equity granted \nto new hires at each level  \nGrowth  Company is aggressively

**We can see a difference in the order of the context retrieved after applying LongContextReorder**

**Setup Filtering and LongContext Retriever Pipeline**

In [38]:
from re import search
filter = EmbeddingsRedundantFilter(embeddings=hf_bge_embeddings)
reordering  = LongContextReorder()
pipeline = DocumentCompressorPipeline(transformers=[filter,reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
    base_compressor=pipeline,base_retriever=lotr,search_kwargs={"k":5,"include_metadata":True})

docs = compression_retriever_reordered.get_relevant_documents("What is esops?")
print(len(docs))

print(docs[0].page_content)

print(compression_retriever_reordered.get_relevant_documents("What is the stigma associated with mental health?")[0].page_content)


10
ESOPS FOR THE LONG TERM  Part IV
Social Impact Considerations  
•Companies focused on social impact goals and/or in developing 
markets may have unique ESOP considerations  
•Consider two possible scenarios:  
–Financial Inclusion Goals : Employee -friendly hiring practices and 
ESOPs can help build local financial inclusion. If this is part of your 
corporate mandate, consider the added social impact value of 
offering your options program to all levels of employees  
–Local Ownership Culture : Conversely, in certain regions stock 
options may have negligible value to employees, either because of 
risk aversion, lack of liquidity, or lack of understanding. If this is the 
case, it is not worth extensively offering options to employees who 
would rather be paid in cash


**Implementing Generation Pipeline with Langchain and HFHub Models**

**Setup LLM**

In [40]:
%%capture
!pip install llama-cpp-python

In [49]:
from langchain.llms import LlamaCpp
llms = LlamaCpp(streaming=True,
                model_path="/content/drive/MyDrive/zephyr-7b-beta.Q4_K_M.gguf",
                max_tokens=2048,
                temperature=0.6,
                top_p=0.95,
                gpu_layers=0,
                stream=True,
                verbose=True,
                n_threads=int(os.cpu_count()/2),
                n_ctx=8192) #context length

/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! gpu_layers is not default parameter.
                gpu_layers was transferred to model_kwargs.
                Please confirm that gpu_layers is what you intended.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  warnings.warn(
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


**Setup RetrievalQA Chain**

In [50]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llms,
    chain_type="stuff",
    retriever= compression_retriever_reordered,
    return_source_documents=True
)

**Querying**

In [ ]:
query = "What is esop?"
results = qa(query)
print(results['result'])

print(results["source_documents"])


In [ ]:
results = qa("How to determine Dollar value of the options Grant")
print(results['result'])

print(results['source_documents'])

for source in results['source_documents']:
  print(source.metadata)